# SCENIC GRN inference on TM - 10X

## Imports

In [ ]:
import os
import glob
import pickle
import pandas as pd
import numpy as np

from dask.diagnostics import ProgressBar

from arboreto.utils import load_tf_names
from arboreto.algo import grnboost2

from pyscenic.rnkdb import FeatherRankingDatabase as RankingDatabase
from pyscenic.utils import modules_from_adjacencies, load_motifs
from pyscenic.prune import prune2df, df2regulons
from pyscenic.aucell import aucell

import seaborn as sns
import scanpy
import scanpy.api as sc

## Load all pseudobulk mouse atlases

In [2]:
droplet = scanpy.read_h5ad('/work/sduknn/Andreas/TM_MCA/TM/droplet_pseudobulk.h5ad')
facs = scanpy.read_h5ad('/work/sduknn/Andreas/TM_MCA/TM/facs_pseudobulk.h5ad')
mca = scanpy.read_h5ad('/work/sduknn/Andreas/TM_MCA/MCA/mca_pseudobulk.h5ad')

In [3]:
#Only use overlap between all datasets
overlap = set(droplet.var_names.values) & set(facs.var_names.values) & set(mca.var_names.values)


droplet = droplet[:, list(overlap)]
facs = facs[:, list(overlap)]
mca = mca[:, list(overlap)]

sc.pp.filter_genes(droplet, min_cells=(len(droplet.obs_names) / 10))
sc.pp.filter_genes(facs, min_cells=(len(facs.obs_names) / 10))
sc.pp.filter_genes(mca, min_cells=(len(mca.obs_names) / 10))

overlap = set(droplet.var_names.values) & set(facs.var_names.values) & set(mca.var_names.values)


droplet = droplet[:, list(overlap)]
facs = facs[:, list(overlap)]
mca = mca[:, list(overlap)]

In [4]:
#library size normalization
sc.pp.normalize_per_cell(droplet, counts_per_cell_after=1e4)


#log transform
sc.pp.log1p(droplet)


## Settings for SCENIC 

In [2]:
#Settings
DATA_FOLDER="/work/sduknn/Andreas/TM_MCA/database_SCENIC/tmp/droplet"
RESOURCES_FOLDER="/work/sduknn/Andreas/TM_MCA/database_SCENIC/resources"
DATABASE_FOLDER = "/work/sduknn/Andreas/TM_MCA/database_SCENIC/databases/"

#For clusters, probably wont have to use
SCHEDULER="172.24.5.14:8786"

DATABASES_GLOB = os.path.join(DATABASE_FOLDER, "mm9-*.feather")
MOTIF_ANNOTATIONS_FNAME = os.path.join(RESOURCES_FOLDER, "motifs-v9-nr.mgi-m0.001-o0.0.tbl")
MM_TFS_FNAME = os.path.join(RESOURCES_FOLDER, '/work/sduknn/Andreas/TM_MCA/database_SCENIC/making_TF_file/mm_tfs.txt')
#SC_EXP_FNAME = os.path.join(RESOURCES_FOLDER, "PATH.txt")
REGULONS_FNAME = os.path.join(DATA_FOLDER, "regulons_10_percent.p")
MOTIFS_FNAME = os.path.join(DATA_FOLDER, "motifs_10_percent.csv")



## Expression matrix as a pandas DataFrame

In [11]:
#Transpose and check array
ex_matrix = droplet.X
ex_matrix.shape

(1061, 11245)

In [12]:
#make pandas dataframe
ex_matrix_drop = pd.DataFrame(data= ex_matrix.todense(),
                         index=pd.DataFrame(droplet.obs.index.values),
                         columns=droplet.var.index.values)

## Load databases

In [14]:
#Load the TF file
tf_names = load_tf_names(MM_TFS_FNAME)

In [15]:
#Load the ranking databases
db_fnames = glob.glob(DATABASES_GLOB)
def name(fname):
    return os.path.basename(fname).split(".")[0]
dbs = [RankingDatabase(fname=fname, name=name(fname)) for fname in db_fnames]
dbs

dbs2= [dbs[2],dbs[4] ]
dbs2

[FeatherRankingDatabase(name="mm9-500bp-upstream-7species"),
 FeatherRankingDatabase(name="mm9-tss-centered-10kb-7species")]

## Setup dask cluster on slurm

In [16]:
from distributed import LocalCluster, Client
from dask_jobqueue import SLURMCluster

In [17]:
custom_client = SLURMCluster(project='xxxxx', cores=24, walltime='02:00:00', memory='50GB',processes=12)

In [18]:
custom_client.scale(120)
client = Client(custom_client)

## Coexpression inference using GRNBoost

In [22]:
%%time
adjacencies_drop = grnboost2(ex_matrix_drop, tf_names=tf_names, verbose=True,  client_or_address=client)

modules_drop = list(modules_from_adjacencies(adjacencies_drop, ex_matrix_drop))

preparing dask client
parsing input
creating dask graph


/home/foenssmoeller/.local/lib/python3.6/site-packages/arboreto/algo.py:214: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  expression_matrix = expression_data.as_matrix()


120 partitions
computing dask graph
not shutting down client, client was created externally
finished


/home/foenssmoeller/.local/lib/python3.6/site-packages/pyscenic/utils.py:138: RuntimeWarning: invalid value encountered in greater
  regulations = (rhos > rho_threshold).astype(int) - (rhos < -rho_threshold).astype(int)
/home/foenssmoeller/.local/lib/python3.6/site-packages/pyscenic/utils.py:138: RuntimeWarning: invalid value encountered in less
  regulations = (rhos > rho_threshold).astype(int) - (rhos < -rho_threshold).astype(int)


CPU times: user 6min 53s, sys: 13.1 s, total: 7min 6s
Wall time: 3min 48s


## Pruning regulons using RCisTarget

In [ ]:
df_drop = prune2df(dbs2, modules_drop, MOTIF_ANNOTATIONS_FNAME,  client_or_address=client)

In [25]:
# Create regulons from this table of enriched motifs.
regulons_drop = df2regulons(df_drop)

In [26]:
# Save the enriched motifs and the discovered regulons to disk.
df_drop.to_csv(MOTIFS_FNAME)
with open(REGULONS_FNAME, "wb") as f:
    pickle.dump(regulons_drop, f)

In [ ]:
custom_client.close()